# 自動機械学習 Automated ML による品質予測モデル構築

製造工程のセンサーデータの情報から、検査工程の前に製造品の品質を予測することで、早めに不良品を検知できます。リワークや廃棄を早めに行うことでコスト削減で期待されます。本Notebookでは、製造工程のサンプルデータ *Factory.csv* を利用し、製造工程のデータから品質を予測する機械学習を構築します。Automated Machine Learning という自動機械学習を利用します。

# 品質の予測とは？

検査工程の前の段階 (例えば全8工程のうち工程3番目)の早い段階で製造品の品質を予測します。早めに不良品が検知できるので、工程4番目以降で使用する部品などのコストを抑えることができますし、早めのリワークによって製造品を廃棄しないことが期待できます。

 ## 簡易イメージ図

<img src="../../docs/QC-Predict.png" width="250" align="left" >    

# Automated Machine Learning 

Automated Machine Learning は、Microsoft Researchが開発したインテリジェントなモデル探索アルゴリズムを用いて、"前処理"、"アルゴリズム選択"、"パラメータ選択"を自動で行います。PowerBIでも同様の機能をノーコーディングでご提供しています。Automated Machine Learning の詳細については[ドキュメント](https://docs.microsoft.com/ja-jp/azure/machine-learning/service/concept-automated-ml)をご参照ください。

<img src="../../docs/AutoML.png" width="600" align="left" >    

## 1. Python SDK のインポート
Azure Machine Learning service の Python SDKをインポートします。

In [13]:
import azureml.core
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.train.automl import AutoMLConfig
from azureml.train.automl.run import AutoMLRun
import os

In [14]:
print("Azure ML SDK Version: ", azureml.core.VERSION)

Azure ML SDK Version:  1.0.23


### Azure ML workspace との接続
Azure Machine Learning service との接続を行います。Azure に対する認証が必要です。

In [15]:
ws = Workspace.from_config()
print(ws.name, ws.location, ws.resource_group, ws.location, sep = '\t')

Found the config file in: /Users/konabuta/Project/Manufacturing-ML/.azureml/config.json
azureml	eastus	dllab	eastus


### 実験名 Experiment 設定

In [16]:
experiment=Experiment(ws, "automl_QC")

# 2. 学習データの準備

In [17]:
import pandas as pd
import os 

df = pd.read_csv('./data/Factory.csv')

In [18]:
df.tail(10)

,ID,Quality,ProcessA-Pressure,ProcessA-Humidity,ProcessA-Vibration,ProcessB-Light,ProcessB-Skill,ProcessB-Temp,ProcessB-Rotation,ProcessC-Density,ProcessC-PH,ProcessC-skewness,ProcessC-Time
4888,4889,0,6.80,0.22,0.36,1.20,0.05,38.00,127.00,0.99,3.04,0.54,9.20
4889,4890,0,4.90,0.23,0.27,11.75,0.03,34.00,118.00,1.00,3.07,0.50,9.40
4890,4891,0,6.10,0.34,0.29,2.20,0.04,25.00,100.00,0.99,3.06,0.44,11.80
4891,4892,0,5.70,0.21,0.32,0.90,0.04,38.00,121.00,0.99,3.24,0.46,10.60
4892,4893,0,6.50,0.23,0.38,1.30,0.03,29.00,112.00,0.99,3.29,0.54,9.70
4893,4894,0,6.20,0.21,0.29,1.60,0.04,24.00,92.00,0.99,3.27,0.50,11.20
4894,4895,0,6.60,0.32,0.36,8.00,0.05,57.00,168.00,0.99,3.15,0.46,9.60
4895,4896,0,6.50,0.24,0.19,1.20,0.04,30.00,111.00,0.99,2.99,0.46,9.40
4896,4897,1,5.50,0.29,0.30,1.10,0.02,20.00,110.00,0.99,3.34,0.38,12.80
4897,4898,0,6.00,0.21,0.38,0.80,0.02,22.00,98.00,0.99,3.26,0.32,11.80


In [19]:
from sklearn.model_selection import train_test_split

X = df.drop(columns=["Quality","ID"],axis=1)
y = df["Quality"].values

X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.1,random_state=100,stratify=y)

# 3. 事前設定

In [20]:
Automl_config = AutoMLConfig(task = 'classification',
                             primary_metric = 'AUC_weighted',
                             iteration_timeout_minutes = 10,
                             iterations = 5,
                             experiment_exit_score = 0.999,
                             blacklist_models = ['KNN'],
                             X = X_train,
                             y = y_train,
                             n_cross_validations = 3)

# 4. 実行と結果確認

In [21]:
local_run = experiment.submit(Automl_config, show_output=True)

Running on local machine
Parent Run ID: AutoML_2ba66d04-d53f-426c-ac71-d5ede4b9dd64
****************************************************************************************************
ITERATION: The iteration being evaluated.
PIPELINE: A summary description of the pipeline being evaluated.
DURATION: Time taken for the current iteration.
METRIC: The result of computing score on the fitted pipeline.
BEST: The best observed score thus far.
****************************************************************************************************

 ITERATION   PIPELINE                                       DURATION      METRIC      BEST
         0   MaxAbsScaler LightGBM                          0:00:22       0.8434    0.8434
         1   RobustScaler LightGBM                          0:00:21       0.8763    0.8763
         2   RobustScaler LogisticRegression                0:00:21       0.7859    0.8763
         3   StandardScalerWrapper LightGBM                 0:00:21       0.8432    0.8763
 

In [22]:
from azureml.widgets import RunDetails
RunDetails(local_run).show()

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

In [23]:
best_run, fitted_model = local_run.get_output()
best_run

Experiment,Id,Type,Status,Details Page,Docs Page
automl_QC_SHAP,AutoML_2ba66d04-d53f-426c-ac71-d5ede4b9dd64_1,,Completed,Link to Azure Portal,Link to Documentation


In [24]:
fitted_model

Pipeline(memory=None,
     steps=[('RobustScaler', RobustScaler(copy=True, quantile_range=[25, 75], with_centering=True,
       with_scaling=False)), ('LightGBMClassifier', <automl.client.core.common.model_wrappers.LightGBMClassifier object at 0x11d2e3320>)])